In [76]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import scan
from bluesky.plan_stubs import mv, open_run, close_run, create, \
    save, drop, rd, trigger, trigger_and_read
from ophyd.sim import det1, noisy_det, motor1
from databroker import Broker
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

db = Broker.named('temp')
RE.subscribe(db.insert)

Tiled version 0.1.0b5


0

In [ ]:
def plan_without_data():
    # Plan procedures
    yield from mv(motor1, 10)

uid = RE(plan_without_data())
print(uid)

In [ ]:
def plan_with_data():
    yield from open_run(md={'content': 'Start Document Additional Metadata'})
    
    # Plan procedures
    yield from mv(motor1, 10)
    
    yield from close_run()

uid = RE(plan_with_data())
print(uid)

## Documents

### Metadata on Standard Plans

In [ ]:
(uid, ) = RE(scan([det1], motor1, -3, 3, 15, md={
    "motive": "Calibration",
    "sample": "Standard Sample"
}))
db[uid]

In [ ]:
last_run = db[-1]
last_run

### Start Event

- time: Horário de início do scan

- plan_name: Nome do plano executado, como 'scan' ou 'grid_scan'

- uid: ID único que identifica o scan realizado

- scan_id: Número inteiro amigável ao usuário, mas não necessáriamente único 

- Outros metadados inseridos pelo usuário ou plano

In [ ]:
last_run.start

### Stop Event

- time: Horário de fim do scan

- exit_status: Status do plano ao final ('abort', 'fail', 'success')

In [ ]:
last_run.stop

### Descriptors

In [ ]:
last_run.descriptors[0]

### Events

In [ ]:
last_run.table()

## Metadata Plan Stubs

In [74]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1].table()



+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 16:58:09.3 |      0.961 |
+-----------+------------+------------+
generator plan_with_data ['e38627ce'] (scan num: 6)




,noisy_det,time
seq_num,,
1,0.961416,2024-09-03 19:58:09.306817770


In [75]:
db[-1]

In [80]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger(noisy_det)
    yield from rd(motor1)

    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1]

generator plan_with_data ['7f438b1d'] (scan num: 4)




In [81]:
def plan_with_data():
    yield from open_run(md={})
    yield from create(name="custom_data_stream")
    
    # Plan procedures
    yield from trigger(noisy_det)
    yield from rd(motor1)

    yield from save()
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1]

generator plan_with_data ['f5d19204'] (scan num: 5)




In [83]:
db[-1].table("custom_data_stream")

,motor1,motor1_setpoint,time
seq_num,,,
1,10.0,10,2024-09-03 20:03:20.070881367


create: Bundle future readings into a new Event document.

save: Close a bundle of readings and emit a completed Event document.

drop: Drop a bundle of readings without emitting a completed Event document.

subscribe: Subscribe the stream of emitted documents.

unsubscribe: Remove a subscription.


Plan Stubs

trigger: Trigger and acquisition.

read: Take a reading and add it to the current bundle of readings.

rd: Reads a single-value non-triggered object

Preprocessors -> What is a decorator

relative_set_decorator: Interpret 'set' messages on devices as relative to initial position.

reset_positions_decorator: Return movable devices to their initial positions at the end.

run_decorator: Enclose in 'open_run' and 'close_run' messages.

stage_decorator: 'Stage' devices (i.e., prepare them for use, 'arm' them) and then unstage.

finalize_decorator: try...finally helper

Suplemental Data



baseline_wrapper()

monitor_during_wrapper()

fly_during_wrapper()